In [156]:
import os
import random
import re
import csv
import pandas as pd
from enum import IntEnum
from IPython.display import clear_output
from mahjong.shanten import Shanten

In [157]:
#調整所需資料的參數
jumpWords = ['UN'] #某個牌局中，不明原因在打牌階段出現的詞。會導致程式停止
year = 2021 #要處理的年份
writed_file = 'DiscardData.csv'#要寫入的檔名
feature_Name = 'discard_feature.xlsx' #要使用的標籤名
features_nums = 987 #讀取的features數量
chow_data_num = 0
pung_data_num = 0
kong_data_num = 0
riichi_data_num = 0
record_games = 1000 #要處理的遊戲局數

In [158]:
#轉換資料所需暫存變數
class PlayState(IntEnum):
    INIT = 0
    PLAYING = 1
    WIN = 2
    DRAW = 3

class EventState(IntEnum):
    CONTINUE = 0
    RICHII = 1
    N = 2
    DORA = 3
    INIT_DORA = 4

class GameInfo:
    def __init__(self):
        self.reset()
    
    def reset(self):
        self.playState = PlayState.INIT
        self.eventState = EventState.CONTINUE
        self.startPlayer = 0
        self.eventPlayer = -1 #發生事件的玩家
        self.windlist = [0,1,2,3]#0:東 1:南 2:西 3:北 
        self.Field_wind = 0
        self.score = [250,250,250,250]
        self.dora = [-1,-1,-1,-1,-1]
        #手牌
        self.hai0 = []
        self.hai1 = []
        self.hai2 = []
        self.hai3 = []
        #副露
        self.N0 = []
        self.N1 = []
        self.N2 = []
        self.N3 = []
        self.rounds = 0
        self.play0Dis = []
        self.play1Dis = []
        self.play2Dis = []
        self.play3Dis = []
        self.IfREACHPlayer = [0,0,0,0] 
        self.IfPlayerChow = [0,0,0,0]
        self.IfPlayerPung = [0,0,0,0]
        self.IfPlayerKong = [0,0,0,0]
        self.datalist = []#一次寫入csv的資料單位
        self.rowData = [0]*features_nums #每一行資料
        self.chowFlag = 0
        self.pungFlag = 0
        self.kongFlag = 0
        self.riichiFlag = 0
        #新增特徵資訊
        self.remain_tile_count = [4]*34
        self.last_discard_134 = -1
#注意 剛加入還是可以透過rowData改datalist的值

In [ ]:
"""
常用函式宣告
"""
def intTo16bitBinary(x: int):#int轉2進制16位
    binaryStr = ''
    for i in range(15,-1,-1):
        bit = (x >> i) & 1
        binaryStr += str(bit)
    return binaryStr

def Binary16bitToInt(str_x:str,bits):#2進制16位轉int
    x_bits = str_x[:bits]
    decimal_value = int(x_bits, 2)
    return decimal_value

def tileNumberTrans(playerHai,templist):#將136張牌轉換成34種
    templist = [0] * 34
    for tile in playerHai:
        templist[int((tile/4))] = templist[int((tile/4))] + 1
    return templist

def cpoylist(game_info:GameInfo,RowIndex:int,copyedlist:list):#輸入rowData下一個空的index，把copyedlist的東西加到後面
    for data in copyedlist:
        game_info.rowData[RowIndex] = data
        RowIndex = RowIndex + 1
    return game_info

def INIT_Dora(game_info: GameInfo, element: str):
    game_info.eventState = EventState.CONTINUE
    number_str = element.split('=')[1].strip('"')
    numbers = number_str.split(',')
    game_info.Field_wind = int(numbers[0])
    last_value = numbers[-1]
    DORAhai = int(int(last_value) / 4)
    for i in range(len(game_info.dora)):
        if(game_info.dora[i] == -1):
            game_info.dora[i] = DORAhai
            game_info.remain_tile_count[DORAhai] -= 1
            break
    return game_info

def Switch_StartPlayer_And_Selfwind(game_info: GameInfo, element: str):
    game_info.startPlayer = int(element[5])
    for i in range(len(game_info.windlist)):
        game_info.windlist[i] = (game_info.windlist[i] + 4 - game_info.startPlayer) % 4
    return game_info

def Process_Score(game_info: GameInfo, element: str):
    tenlist = re.split(r'[",]', element)
    tenlist = [s for s in tenlist if s]
    del tenlist[0]
    for i in range(len(game_info.score)):
        game_info.score[i] = int(tenlist[i])
    return game_info

def INIT_Hai(game_info: GameInfo, element: str,random_player:int):

    if('hai0' in element):
        game_info.hai0 = haitrans(element)
        if random_player == 0:
            for tile in game_info.hai0:
                game_info.remain_tile_count[int(tile/4)] -= 1
    elif('hai1' in element):
        game_info.hai1 = haitrans(element)
        if random_player == 1:
            for tile in game_info.hai1:
                game_info.remain_tile_count[int(tile/4)] -= 1
    elif('hai2' in element):
        game_info.hai2 = haitrans(element)
        if random_player == 2:
            for tile in game_info.hai2:
                game_info.remain_tile_count[int(tile/4)] -= 1
    elif('hai3' in element):
        game_info.hai3 = haitrans(element)
        game_info.playState = PlayState.PLAYING
        if random_player == 3:
            for tile in game_info.hai3:
                game_info.remain_tile_count[int(tile/4)] -= 1
    return game_info

def haitrans(strlist):#轉換手牌
    strHailist = re.split(r'[",]', strlist)
    strHailist = [s for s in strHailist if s]
    del strHailist[0]
    intHailist = []
    for tile in strHailist:
        intHailist.append(int(tile))
    return intHailist

def Process_Playing(game_info: GameInfo, element: str, random_Player: int):
    #這裡的int(element[1:])都未除以4
    if(element == 'DORA'):
        game_info.eventState = EventState.DORA
    elif(game_info.eventState == EventState.DORA):
        tmpDORAhai = int(int(element.split('=')[1].strip('"'))/4)
        for i in range(len(game_info.dora)):
            if(game_info.dora[i] == -1):
                game_info.dora[i] = tmpDORAhai
                game_info.remain_tile_count[tmpDORAhai] -= 1
                break
        game_info.eventState = 0
    elif(element == 'AGARI'):
        game_info.playState = PlayState.WIN
        game_info.eventState = EventState.CONTINUE
    elif(element == 'RYUUKYOKU'):
        game_info.playState = PlayState.DRAW
        game_info.eventState = EventState.CONTINUE
    elif(element == 'REACH'):
        #game_info = record_riichi_label(game_info,1)
        game_info.eventState = EventState.RICHII
    elif(game_info.eventState == EventState.RICHII):
        if('who' in element):
            game_info.eventPlayer = int(element[5])
            game_info.IfREACHPlayer[game_info.eventPlayer] = 1
        elif('ten' in element):
            tenlist = re.split(r'[",]', element)
            tenlist = [s for s in tenlist if s]
            del tenlist[0]
            for i in range(len(game_info.score)):
                game_info.score[i] = int(tenlist[i])
        elif('step' in element and element[6] == '2'):
            game_info.eventPlayer = -1
            game_info.eventState = 0
    elif(element == 'N'):
        game_info.eventState = EventState.N
    elif(game_info.eventState == EventState.N and game_info.eventPlayer == -1):
        game_info.eventPlayer = int(element[5])
    elif(game_info.eventState == EventState.N and game_info.eventPlayer != -1):
        Ntype = int(element.split('=')[1].strip('"'))
        binary16Bit = intTo16bitBinary(Ntype)
        if(binary16Bit[13] == '1'):
            #game_info = record_chow_label(game_info,1)
            game_info = chow(binary16Bit,game_info,random_Player)
        elif(binary16Bit[12] == '1'):
            #game_info = record_pung_label(game_info,1)
            #game_info.chowFlag = 0
            game_info = pung(binary16Bit,game_info,random_Player)
        else:
            #game_info.chowFlag = 0
            #game_info = record_kong_label(game_info,1)
            game_info = kong(binary16Bit,game_info,random_Player)
        game_info.eventState = 0
        game_info.eventPlayer = -1
    elif(element[0] == 'T'):
        #game_info = record_pung_label(game_info,0)
        #game_info = record_kong_label(game_info,0)
        #game_info = record_chow_label(game_info,0)
        game_info.rounds = game_info.rounds + 1
        game_info.hai0.append(int(element[1:]))
        if random_Player == 0:
            game_info.remain_tile_count[int(int(element[1:])/4)] -= 1#減掉摸到的牌
        #if check_if_can_Riichi(hai136_to_hai34(game_info.hai0),hai136_to_hai34(game_info.N0)):
            #game_info = record_riichi_info(game_info,0)
        #game_info = record_kong_info(int(element[1:]),check_if_Draw_can_kong(int(element[1:]), 0, game_info), game_info)
    elif(element[0] == 'D'):
        #game_info = record_kong_label(game_info,0)
        #game_info = record_riichi_label(game_info,0)
        record_discard_info(int(element[1:]), game_info, 0,random_Player)
        game_info.hai0.remove(int(element[1:]))
        game_info.play0Dis.append(int(element[1:]))
        game_info.last_discard_134 = int(element[1:])
        if random_Player != 0:
            game_info.remain_tile_count[int(int(element[1:])/4)] -= 1#減掉棄的牌
        #if(check_if_can_chow(int(element[1:]),game_info.hai1)):
            #game_info = record_chow_info(int(element[1:]),game_info,1)
        #game_info = record_pung_info(int(element[1:]),check_if_can_pung(int(element[1:]), 0, game_info), game_info)
        #game_info = record_kong_info(int(element[1:]),check_if_Discard_can_kong(int(element[1:]), 0, game_info), game_info)
        #if random_Player == 0:
            #game_info = record_predictor_info(game_info,random_Player)
    elif(element[0] == 'U'):
        #game_info = record_pung_label(game_info,0)
        #game_info = record_kong_label(game_info,0)
        #game_info = record_chow_label(game_info,0)
        game_info.rounds = game_info.rounds + 1
        game_info.hai1.append(int(element[1:]))
        if random_Player == 1:
            game_info.remain_tile_count[int(int(element[1:])/4)] -= 1#減掉摸到的牌
        #if check_if_can_Riichi(hai136_to_hai34(game_info.hai1),hai136_to_hai34(game_info.N1)):
            #game_info = record_riichi_info(game_info,1)
        #game_info = record_kong_info(int(element[1:]),check_if_Draw_can_kong(int(element[1:]), 1, game_info), game_info)
    elif(element[0] == 'E'):
        #game_info = record_kong_label(game_info,0)
        #game_info = record_riichi_label(game_info,0)
        record_discard_info(int(element[1:]), game_info, 1,random_Player)
        game_info.hai1.remove(int(element[1:]))
        game_info.play1Dis.append(int(element[1:]))
        game_info.last_discard_134 = int(element[1:])
        if random_Player != 1:
            game_info.remain_tile_count[int(int(element[1:])/4)] -= 1#減掉棄的牌
        #if(check_if_can_chow(int(element[1:]),game_info.hai2)):
            #game_info = record_chow_info(int(element[1:]),game_info,2)
        #game_info = record_pung_info(int(element[1:]),check_if_can_pung(int(element[1:]), 1, game_info), game_info)
        #game_info = record_kong_info(int(element[1:]),check_if_Discard_can_kong(int(element[1:]), 1, game_info), game_info)
        #if random_Player == 1:
            #game_info = record_predictor_info(game_info,random_Player)
    elif(element[0] == 'V'):
        #game_info = record_pung_label(game_info,0)
        #game_info = record_kong_label(game_info,0)
        #game_info = record_chow_label(game_info,0)
        game_info.rounds = game_info.rounds + 1
        game_info.hai2.append(int(element[1:]))
        if random_Player == 2:
            game_info.remain_tile_count[int(int(element[1:])/4)] -= 1#減掉摸到的牌
        #if check_if_can_Riichi(hai136_to_hai34(game_info.hai2),hai136_to_hai34(game_info.N2)):
            #game_info = record_riichi_info(game_info,2)
        #game_info = record_kong_info(int(element[1:]),check_if_Draw_can_kong(int(element[1:]), 2, game_info), game_info)
    elif(element[0] == 'F'):
        #game_info = record_kong_label(game_info,0)
        #game_info = record_riichi_label(game_info,0)
        record_discard_info(int(element[1:]), game_info, 2,random_Player)
        game_info.hai2.remove(int(element[1:]))
        game_info.play2Dis.append(int(element[1:]))
        game_info.last_discard_134 = int(element[1:])
        if random_Player != 2:
            game_info.remain_tile_count[int(int(element[1:])/4)] -= 1#減掉棄的牌
        #if(check_if_can_chow(int(element[1:]),game_info.hai3)):
            #game_info = record_chow_info(int(element[1:]),game_info,3)
        #game_info = record_pung_info(int(element[1:]),check_if_can_pung(int(element[1:]), 2, game_info), game_info)
        #game_info = record_kong_info(int(element[1:]),check_if_Discard_can_kong(int(element[1:]), 2, game_info), game_info)
        #if random_Player == 2:
            #game_info = record_predictor_info(game_info,random_Player)
    elif(element[0] == 'W'):
        #game_info = record_pung_label(game_info,0)
        #game_info = record_kong_label(game_info,0)
        #game_info = record_chow_label(game_info,0)
        game_info.rounds = game_info.rounds + 1
        game_info.hai3.append(int(element[1:]))
        if random_Player == 3:
            game_info.remain_tile_count[int(int(element[1:])/4)] -= 1#減掉摸到的牌
        #if check_if_can_Riichi(hai136_to_hai34(game_info.hai3),hai136_to_hai34(game_info.N3)):
            #game_info = record_riichi_info(game_info,3)
        #game_info = record_kong_info(int(element[1:]),check_if_Draw_can_kong(int(element[1:]), 3, game_info), game_info)
    elif(element[0] == 'G'):
        #game_info = record_kong_label(game_info,0)
        #game_info = record_riichi_label(game_info,0)
        record_discard_info(int(element[1:]), game_info, 3,random_Player)
        game_info.hai3.remove(int(element[1:]))
        game_info.play3Dis.append(int(element[1:]))
        game_info.last_discard_134 = int(element[1:])
        if random_Player != 3:
            game_info.remain_tile_count[int(int(element[1:])/4)] -= 1#減掉棄的牌
        #if(check_if_can_chow(int(element[1:]),game_info.hai0)):
            #game_info = record_chow_info(int(element[1:]),game_info,0)
        #game_info = record_pung_info(int(element[1:]),check_if_can_pung(int(element[1:]), 3, game_info), game_info)
        #game_info = record_kong_info(int(element[1:]),check_if_Discard_can_kong(int(element[1:]), 3, game_info), game_info)
        #if random_Player == 3:
            #game_info = record_predictor_info(game_info,random_Player)
    return game_info

def chow(binary16Bit: str,game_info: GameInfo,random_Player:int):
    chowNumber = Binary16bitToInt(binary16Bit,6)
    tile_list = [int(binary16Bit[11:13],2),int(binary16Bit[9:11],2),int(binary16Bit[7:9],2)]#紀錄小到大數牌是第幾張
    tileType = int(chowNumber / 21) #0:character 1:circle 2:bamboo
    chowType = int(chowNumber / 3) % 7#0~6張牌
    chowTile = chowNumber % 3 #吃的那張牌是哪張 0最小 1中間 2最大
    smallTile_34 =  tileType * 9 + chowType #吃的最小牌
    game_info.IfPlayerChow[game_info.eventPlayer] = 1
    if(game_info.eventPlayer == 0):
        game_info.N0.append(smallTile_34)
        game_info.N0.append((smallTile_34+1))
        game_info.N0.append((smallTile_34+2))
        for i in range(3):
            if i == chowTile:
                continue
            game_info.hai0.remove((smallTile_34+i)*4+tile_list[i])
        #減掉剩餘牌數
        if random_Player != 0:
            game_info.remain_tile_count[smallTile_34] -= 1
            game_info.remain_tile_count[smallTile_34+1] -= 1
            game_info.remain_tile_count[smallTile_34+2] -= 1
            game_info.remain_tile_count[smallTile_34 + chowTile] += 1#棄牌的時候減過了
    elif(game_info.eventPlayer == 1):
        game_info.N1.append(smallTile_34)
        game_info.N1.append((smallTile_34+1))
        game_info.N1.append((smallTile_34+2))
        for i in range(3):
            if i == chowTile:
                continue
            game_info.hai1.remove((smallTile_34+i)*4+tile_list[i])
        #減掉剩餘牌數
        if random_Player != 1:
            game_info.remain_tile_count[smallTile_34] -= 1
            game_info.remain_tile_count[smallTile_34+1] -= 1
            game_info.remain_tile_count[smallTile_34+2] -= 1
            game_info.remain_tile_count[smallTile_34 + chowTile] += 1#棄牌的時候減過了
    elif(game_info.eventPlayer == 2):
        game_info.N2.append(smallTile_34)
        game_info.N2.append((smallTile_34+1))
        game_info.N2.append((smallTile_34+2))
        for i in range(3):
            if i == chowTile:
                continue
            game_info.hai2.remove((smallTile_34+i)*4+tile_list[i])
        #減掉剩餘牌數
        if random_Player != 2:
            game_info.remain_tile_count[smallTile_34] -= 1
            game_info.remain_tile_count[smallTile_34+1] -= 1
            game_info.remain_tile_count[smallTile_34+2] -= 1
            game_info.remain_tile_count[smallTile_34 + chowTile] += 1#棄牌的時候減過了
    elif(game_info.eventPlayer == 3):
        game_info.N3.append(smallTile_34)
        game_info.N3.append((smallTile_34+1))
        game_info.N3.append((smallTile_34+2))
        for i in range(3):
            if i == chowTile:
                continue
            game_info.hai3.remove((smallTile_34+i)*4+tile_list[i])
        #減掉剩餘牌數
        if random_Player != 3:
            game_info.remain_tile_count[smallTile_34] -= 1
            game_info.remain_tile_count[smallTile_34+1] -= 1
            game_info.remain_tile_count[smallTile_34+2] -= 1
            game_info.remain_tile_count[smallTile_34 + chowTile] += 1#棄牌的時候減過了
    return game_info
    
def pung(binary16Bit: str,game_info: GameInfo,random_Player:int):
    pungNumber = Binary16bitToInt(binary16Bit,7)
    tileNumber = int(pungNumber / 3)
    game_info.IfPlayerPung[game_info.eventPlayer] = 1
    if(game_info.eventPlayer == 0):
        game_info.hai0.append(game_info.last_discard_134)#先將碰的牌來源加進來，下面會刪掉
        game_info.N0.append(tileNumber)
        game_info.N0.append(tileNumber)
        game_info.N0.append(tileNumber)
        #從手牌移除牌
        for i in range(4):
            if i == int(binary16Bit[9:11],2):
                continue
            game_info.hai0.remove(tileNumber * 4 + i)

        #減掉剩餘牌數
        if random_Player != 0:
            game_info.remain_tile_count[tileNumber] -= 2
    elif(game_info.eventPlayer == 1):
        game_info.hai1.append(game_info.last_discard_134)#先將碰的牌來源加進來，下面會刪掉
        game_info.N1.append(tileNumber)
        game_info.N1.append(tileNumber)
        game_info.N1.append(tileNumber)
        #從手牌移除牌
        for i in range(4):
            if i == int(binary16Bit[9:11],2):
                continue
            game_info.hai1.remove(tileNumber * 4 + i)
        #減掉剩餘牌數
        if random_Player != 1:
            game_info.remain_tile_count[tileNumber] -= 2
    elif(game_info.eventPlayer == 2):
        game_info.hai2.append(game_info.last_discard_134)#先將碰的牌來源加進來，下面會刪掉
        game_info.N2.append(tileNumber)
        game_info.N2.append(tileNumber)
        game_info.N2.append(tileNumber)
        #從手牌移除牌
        for i in range(4):
            if i == int(binary16Bit[9:11],2):
                continue
            game_info.hai2.remove(tileNumber * 4 + i)

        #減掉剩餘牌數
        if random_Player != 2:
            game_info.remain_tile_count[tileNumber] -= 2
    elif(game_info.eventPlayer == 3):
        game_info.hai3.append(game_info.last_discard_134)#先將碰的牌來源加進來，下面會刪掉
        game_info.N3.append(tileNumber)
        game_info.N3.append(tileNumber)
        game_info.N3.append(tileNumber)
        #從手牌移除牌
        for i in range(4):
            if i == int(binary16Bit[9:11],2):
                continue
            game_info.hai3.remove(tileNumber * 4 + i)
        #減掉剩餘牌數
        if random_Player != 3:
            game_info.remain_tile_count[tileNumber] -= 2
    return game_info
    
def kong(binary16Bit: str,game_info: GameInfo,random_Player:int):
    game_info.IfPlayerKong[game_info.eventPlayer] = 1
    if(binary16Bit[14] == '0' and binary16Bit[15] == '0'): #暗槓
        kongNumber = Binary16bitToInt(binary16Bit,8)
        tileNumber = int(kongNumber / 4)
        if(game_info.eventPlayer == 0):
            game_info.N0.append(tileNumber)
            game_info.N0.append(tileNumber)
            game_info.N0.append(tileNumber)
            game_info.N0.append(tileNumber)
            game_info.hai0.remove(tileNumber)
            game_info.hai0.remove(tileNumber)
            game_info.hai0.remove(tileNumber)
            game_info.hai0.remove(tileNumber)
            #減掉剩餘牌數
            if random_Player != 0:
                game_info.remain_tile_count[tileNumber] -= 4
        elif(game_info.eventPlayer == 1):
            game_info.N1.append(tileNumber)
            game_info.N1.append(tileNumber)
            game_info.N1.append(tileNumber)
            game_info.N1.append(tileNumber)
            game_info.hai1.remove(tileNumber)
            game_info.hai1.remove(tileNumber)
            game_info.hai1.remove(tileNumber)
            game_info.hai1.remove(tileNumber)
            #減掉剩餘牌數
            if random_Player != 1:
                game_info.remain_tile_count[tileNumber] -= 4
        elif(game_info.eventPlayer == 2):
            game_info.N2.append(tileNumber)
            game_info.N2.append(tileNumber)
            game_info.N2.append(tileNumber)
            game_info.N2.append(tileNumber)
            game_info.hai2.remove(tileNumber)
            game_info.hai2.remove(tileNumber)
            game_info.hai2.remove(tileNumber)
            game_info.hai2.remove(tileNumber)
            #減掉剩餘牌數
            if random_Player != 2:
                game_info.remain_tile_count[tileNumber] -= 4
        elif(game_info.eventPlayer == 3):
            game_info.N3.append(tileNumber)
            game_info.N3.append(tileNumber)
            game_info.N3.append(tileNumber)
            game_info.N3.append(tileNumber)
            game_info.hai3.remove(tileNumber)
            game_info.hai3.remove(tileNumber)
            game_info.hai3.remove(tileNumber)
            game_info.hai3.remove(tileNumber)
            #減掉剩餘牌數
            if random_Player != 3:
                game_info.remain_tile_count[tileNumber] -= 4
    elif(binary16Bit[8:14]=="000000"):#明槓
        kongNumber = Binary16bitToInt(binary16Bit,8)
        tileNumber = int(kongNumber / 4)

        if(game_info.eventPlayer == 0):
            game_info.N0.append(tileNumber)
            game_info.N0.append(tileNumber)
            game_info.N0.append(tileNumber)
            game_info.N0.append(tileNumber)
            
            for i in range(4):
                if tileNumber*4+i == kongNumber:
                    continue
                game_info.hai0.remove(tileNumber*4+i)
            #減掉剩餘牌數
            if random_Player != 0:
                game_info.remain_tile_count[tileNumber] -= 3
        elif(game_info.eventPlayer == 1):
            game_info.N1.append(tileNumber)
            game_info.N1.append(tileNumber)
            game_info.N1.append(tileNumber)
            game_info.N1.append(tileNumber)
            for i in range(4):
                if tileNumber*4+i == kongNumber:
                    continue
                game_info.hai1.remove(tileNumber*4+i)
            #減掉剩餘牌數
            if random_Player != 1:
                game_info.remain_tile_count[tileNumber] -= 3
        elif(game_info.eventPlayer == 2):
            game_info.N2.append(tileNumber)
            game_info.N2.append(tileNumber)
            game_info.N2.append(tileNumber)
            game_info.N2.append(tileNumber)
            for i in range(4):
                if tileNumber*4+i == kongNumber:
                    continue
                game_info.hai2.remove(tileNumber*4+i)
            #減掉剩餘牌數
            if random_Player != 2:
                game_info.remain_tile_count[tileNumber] -= 3
        elif(game_info.eventPlayer == 3):
            game_info.N3.append(tileNumber)
            game_info.N3.append(tileNumber)
            game_info.N3.append(tileNumber)
            game_info.N3.append(tileNumber)
            for i in range(4):
                if tileNumber*4+i == kongNumber:
                    continue
                game_info.hai3.remove(tileNumber*4+i)
            #減掉剩餘牌數
            if random_Player != 3:
                game_info.remain_tile_count[tileNumber] -= 3
    else:#加槓
        pungNumber = Binary16bitToInt(binary16Bit,7)
        kongtile34 = int(pungNumber / 3)
        kongtile136 = kongtile34*4 + int(binary16Bit[9:11],2)

        if(game_info.eventPlayer == 0):
            game_info.N0.append(kongtile34)
            
            game_info.hai0.remove(kongtile136)
            #減掉剩餘牌數
            if random_Player != 0:
                game_info.remain_tile_count[kongtile34] -= 1
        elif(game_info.eventPlayer == 1):
            game_info.N1.append(kongtile34)
            
            game_info.hai1.remove(kongtile136)
            #減掉剩餘牌數
            if random_Player != 1:
                game_info.remain_tile_count[kongtile34] -= 1
        elif(game_info.eventPlayer == 2):
            game_info.N2.append(kongtile34)
            
            game_info.hai2.remove(kongtile136)
            #減掉剩餘牌數
            if random_Player != 2:
                game_info.remain_tile_count[kongtile34] -= 1
        elif(game_info.eventPlayer == 3):
            game_info.N3.append(kongtile34)
            
            game_info.hai3.remove(kongtile136)
            #減掉剩餘牌數
            if random_Player != 3:
                game_info.remain_tile_count[kongtile34] -= 1
    return game_info

def Process_Win(game_info: GameInfo, element: str, player_pos: int, writer):
    if('sc' in element):
        sclist = re.split(r'[",]', element)
        scnlist = [s for s in sclist if s]
        del scnlist[0]
        for i in range(len(game_info.score)):
            game_info.score[i] = int(scnlist[i * 2]) + int(scnlist[i * 2 + 1])
        #game_info = record_predictor_label(game_info,player_pos)
        game_info = write_to_excel(game_info, player_pos, writer)
    return game_info

def Process_Draw(game_info: GameInfo, element: str, player_pos: int, writer):
    if('sc' in element):
        sclist = re.split(r'[",]', element)
        scnlist = [s for s in sclist if s]
        del scnlist[0]
        for i in range(len(game_info.score)):
            game_info.score[i] = int(scnlist[i * 2]) + int(scnlist[i * 2 + 1])
        #game_info = record_predictor_label(game_info,player_pos)
        game_info = write_to_excel(game_info ,player_pos, writer)
    return game_info
        
def write_to_excel(game_info: GameInfo, player_pos: int,writer):#輸入玩家視角，寫入excel
    writer.writerows(game_info.datalist)
    game_info.datalist = []
    return game_info

def record_predictor_label(game_info: GameInfo, player_pos: int):
    for row in game_info.datalist:
        row[-1] = game_info.score[player_pos]
    return game_info


def record_predictor_info(game_info: GameInfo, player_pos: int):
    game_info.rowData = [0]*features_nums
    game_info.rowData[0] = game_info.Field_wind
    game_info.rowData[1] = game_info.windlist[player_pos]
    game_info.rowData[2] = game_info.rounds

    self_Hai_34_List = [0] * 34
    right_Hai_34_list = [0] * 34
    Opposite_Hai_34_list = [0] * 34
    Left_Hai_34_list = [0] * 34
    self_Dis_34_List = [0] * 34
    right_Dis_34_list = [0] * 34
    Opposite_Dis_34_list = [0] * 34
    Left_Dis_34_list = [0] * 34

    if player_pos == 0:
        self_Hai_34_List = tileNumberTrans(game_info.hai0, self_Hai_34_List)
        right_Hai_34_list = tileNumberTrans(game_info.N1, right_Hai_34_list)
        Opposite_Hai_34_list = tileNumberTrans(game_info.N2, Opposite_Hai_34_list)
        Left_Hai_34_list = tileNumberTrans(game_info.N3, Left_Hai_34_list)
        self_Dis_34_List = tileNumberTrans(game_info.play0Dis, self_Dis_34_List)
        right_Dis_34_list = tileNumberTrans(game_info.play1Dis, right_Dis_34_list)
        Opposite_Dis_34_list = tileNumberTrans(game_info.play2Dis, Opposite_Dis_34_list)
        Left_Dis_34_list = tileNumberTrans(game_info.play3Dis, Left_Dis_34_list)
    elif player_pos == 1:
        self_Hai_34_List = tileNumberTrans(game_info.hai1, self_Hai_34_List)
        right_Hai_34_list = tileNumberTrans(game_info.N2, right_Hai_34_list)
        Opposite_Hai_34_list = tileNumberTrans(game_info.N3, Opposite_Hai_34_list)
        Left_Hai_34_list = tileNumberTrans(game_info.N0, Left_Hai_34_list)
        self_Dis_34_List = tileNumberTrans(game_info.play1Dis, self_Dis_34_List)
        right_Dis_34_list = tileNumberTrans(game_info.play2Dis, right_Dis_34_list)
        Opposite_Dis_34_list = tileNumberTrans(game_info.play3Dis, Opposite_Dis_34_list)
        Left_Dis_34_list = tileNumberTrans(game_info.play0Dis, Left_Dis_34_list)
    elif player_pos == 2:
        self_Hai_34_List = tileNumberTrans(game_info.hai2, self_Hai_34_List)
        right_Hai_34_list = tileNumberTrans(game_info.N3, right_Hai_34_list)
        Opposite_Hai_34_list = tileNumberTrans(game_info.N0, Opposite_Hai_34_list)
        Left_Hai_34_list = tileNumberTrans(game_info.N1, Left_Hai_34_list)
        self_Dis_34_List = tileNumberTrans(game_info.play2Dis, self_Dis_34_List)
        right_Dis_34_list = tileNumberTrans(game_info.play3Dis, right_Dis_34_list)
        Opposite_Dis_34_list = tileNumberTrans(game_info.play0Dis, Opposite_Dis_34_list)
        Left_Dis_34_list = tileNumberTrans(game_info.play1Dis, Left_Dis_34_list)
    elif player_pos == 3:
        self_Hai_34_List = tileNumberTrans(game_info.hai3, self_Hai_34_List)
        right_Hai_34_list = tileNumberTrans(game_info.N0, right_Hai_34_list)
        Opposite_Hai_34_list = tileNumberTrans(game_info.N1, Opposite_Hai_34_list)
        Left_Hai_34_list = tileNumberTrans(game_info.N2, Left_Hai_34_list)
        self_Dis_34_List = tileNumberTrans(game_info.play3Dis, self_Dis_34_List)
        right_Dis_34_list = tileNumberTrans(game_info.play0Dis, right_Dis_34_list)
        Opposite_Dis_34_list = tileNumberTrans(game_info.play1Dis, Opposite_Dis_34_list)
        Left_Dis_34_list = tileNumberTrans(game_info.play2Dis, Left_Dis_34_list)
    
    game_info = cpoylist(game_info,3,self_Hai_34_List)
    game_info = cpoylist(game_info,37,self_Dis_34_List)
    game_info.rowData[71] = game_info.score[player_pos]
    game_info = cpoylist(game_info,72,right_Hai_34_list)
    game_info = cpoylist(game_info,106,right_Dis_34_list)
    game_info.rowData[140] = game_info.score[(player_pos + 1) % 4]
    game_info = cpoylist(game_info,141,Opposite_Hai_34_list)
    game_info = cpoylist(game_info,175,Opposite_Dis_34_list)
    game_info.rowData[209] = game_info.score[(player_pos + 2) % 4]
    game_info = cpoylist(game_info,210,Left_Hai_34_list)
    game_info = cpoylist(game_info,244,Left_Dis_34_list)
    game_info.rowData[278] = game_info.score[(player_pos + 3) % 4]
    game_info = cpoylist(game_info,279,game_info.IfREACHPlayer)
    game_info = cpoylist(game_info,283,game_info.dora)
    game_info.datalist.append(game_info.rowData)
    return game_info
    

def record_discard_info(discard: int, game_info: GameInfo, player_pos: int, random_player: int):
    if random_player != player_pos:
        return game_info
    
    game_info.rowData = [0]*features_nums
    self_Hai_34_List = [0] * 34
    right_Hai_34_list = [0] * 34
    Opposite_Hai_34_list = [0] * 34
    Left_Hai_34_list = [0] * 34

    if player_pos == 0:
        self_Hai_34_List = tileNumberTrans(game_info.hai0, self_Hai_34_List)
        right_Hai_34_list = tileNumberTrans(game_info.N1, right_Hai_34_list)
        Opposite_Hai_34_list = tileNumberTrans(game_info.N2, Opposite_Hai_34_list)
        Left_Hai_34_list = tileNumberTrans(game_info.N3, Left_Hai_34_list)
        self_score_34_List = decode_score_to_34(game_info.score[0])
        right_score_34_List = decode_score_to_34(game_info.score[1])
        Opposite_score_34_List = decode_score_to_34(game_info.score[2])
        Left_score_34_List = decode_score_to_34(game_info.score[3])
        shanten_change_34 = calc_shanten_change(hai136_to_hai34(game_info.hai0),discard)
        pon_potential_34 = calc_pon_potential(hai136_to_hai34(game_info.hai0),game_info.remain_tile_count)
        chi_potential_34 =  calc_chi_potential(hai136_to_hai34(game_info.hai0),game_info.remain_tile_count)
    elif player_pos == 1:
        self_Hai_34_List = tileNumberTrans(game_info.hai1, self_Hai_34_List)
        right_Hai_34_list = tileNumberTrans(game_info.N2, right_Hai_34_list)
        Opposite_Hai_34_list = tileNumberTrans(game_info.N3, Opposite_Hai_34_list)
        Left_Hai_34_list = tileNumberTrans(game_info.N0, Left_Hai_34_list)
        self_score_34_List = decode_score_to_34(game_info.score[1])
        right_score_34_List = decode_score_to_34(game_info.score[2])
        Opposite_score_34_List = decode_score_to_34(game_info.score[3])
        Left_score_34_List = decode_score_to_34(game_info.score[0])
        shanten_change_34 = calc_shanten_change(hai136_to_hai34(game_info.hai1),discard)
        pon_potential_34 = calc_pon_potential(hai136_to_hai34(game_info.hai1),game_info.remain_tile_count)
        chi_potential_34 =  calc_chi_potential(hai136_to_hai34(game_info.hai1),game_info.remain_tile_count)
    elif player_pos == 2:
        self_Hai_34_List = tileNumberTrans(game_info.hai2, self_Hai_34_List)
        right_Hai_34_list = tileNumberTrans(game_info.N3, right_Hai_34_list)
        Opposite_Hai_34_list = tileNumberTrans(game_info.N0, Opposite_Hai_34_list)
        Left_Hai_34_list = tileNumberTrans(game_info.N1, Left_Hai_34_list)
        self_score_34_List = decode_score_to_34(game_info.score[2])
        right_score_34_List = decode_score_to_34(game_info.score[3])
        Opposite_score_34_List = decode_score_to_34(game_info.score[0])
        Left_score_34_List = decode_score_to_34(game_info.score[1])
        shanten_change_34 = calc_shanten_change(hai136_to_hai34(game_info.hai2),discard)
        pon_potential_34 = calc_pon_potential(hai136_to_hai34(game_info.hai2),game_info.remain_tile_count)
        chi_potential_34 =  calc_chi_potential(hai136_to_hai34(game_info.hai2),game_info.remain_tile_count)
    elif player_pos == 3:
        self_Hai_34_List = tileNumberTrans(game_info.hai3, self_Hai_34_List)
        right_Hai_34_list = tileNumberTrans(game_info.N0, right_Hai_34_list)
        Opposite_Hai_34_list = tileNumberTrans(game_info.N1, Opposite_Hai_34_list)
        Left_Hai_34_list = tileNumberTrans(game_info.N2, Left_Hai_34_list)
        self_score_34_List = decode_score_to_34(game_info.score[3])
        right_score_34_List = decode_score_to_34(game_info.score[0])
        Opposite_score_34_List = decode_score_to_34(game_info.score[1])
        Left_score_34_List = decode_score_to_34(game_info.score[2])
        shanten_change_34 = calc_shanten_change(hai136_to_hai34(game_info.hai3),discard)
        pon_potential_34 = calc_pon_potential(hai136_to_hai34(game_info.hai2),game_info.remain_tile_count)
        chi_potential_34 =  calc_chi_potential(hai136_to_hai34(game_info.hai2),game_info.remain_tile_count)

    self_Hai_136_List = decode_34_to_136(self_Hai_34_List)
    right_Hai_136_List = decode_34_to_136(right_Hai_34_list)
    Opposite_Hai_136_List = decode_34_to_136(Opposite_Hai_34_list)
    Left_Hai_136_list = decode_34_to_136(Left_Hai_34_list)
    Dora_Hai_34_list = decode_Dora_to_34(game_info.dora)
    Discard_Hai_102_list = decode_3playersDis_to_102(game_info, random_player)
    rounds_34_list = decode_rounds_to_34(game_info.rounds)
    remain_tile_count_34 = game_info.remain_tile_count
    
    game_info = cpoylist(game_info,0,self_Hai_136_List)
    game_info = cpoylist(game_info,136,right_Hai_136_List)
    game_info = cpoylist(game_info,272,Opposite_Hai_136_List)
    game_info = cpoylist(game_info,408,Left_Hai_136_list)
    game_info = cpoylist(game_info,544,Dora_Hai_34_list)
    game_info = cpoylist(game_info,578,Discard_Hai_102_list)
    game_info = cpoylist(game_info,680,self_score_34_List)
    game_info = cpoylist(game_info,714,right_score_34_List)
    game_info = cpoylist(game_info,748,Opposite_score_34_List)
    game_info = cpoylist(game_info,782,Left_score_34_List)
    game_info = cpoylist(game_info,816,rounds_34_list)
    game_info = cpoylist(game_info,850,remain_tile_count_34)
    game_info = cpoylist(game_info,884,shanten_change_34)
    game_info = cpoylist(game_info,918,pon_potential_34)
    game_info = cpoylist(game_info,952,chi_potential_34)


    game_info.rowData[986] = int(discard/4)

    game_info.datalist.append(game_info.rowData)
    return game_info

def record_chow_info(Left_player_discard: int,game_info: GameInfo,player_pos: int):
    game_info.rowData = [0]*features_nums
    self_Hai_34_List = [0] * 34
    right_Hai_34_list = [0] * 34
    Opposite_Hai_34_list = [0] * 34
    Left_Hai_34_list = [0] * 34

    if player_pos == 0:
        self_Hai_34_List = tileNumberTrans(game_info.hai0, self_Hai_34_List)
        right_Hai_34_list = tileNumberTrans(game_info.N1, right_Hai_34_list)
        Opposite_Hai_34_list = tileNumberTrans(game_info.N2, Opposite_Hai_34_list)
        Left_Hai_34_list = tileNumberTrans(game_info.N3, Left_Hai_34_list)
        self_score_34_List = decode_score_to_34(game_info.score[0])
        right_score_34_List = decode_score_to_34(game_info.score[1])
        Opposite_score_34_List = decode_score_to_34(game_info.score[2])
        Left_score_34_List = decode_score_to_34(game_info.score[3])
    elif player_pos == 1:
        self_Hai_34_List = tileNumberTrans(game_info.hai1, self_Hai_34_List)
        right_Hai_34_list = tileNumberTrans(game_info.N2, right_Hai_34_list)
        Opposite_Hai_34_list = tileNumberTrans(game_info.N3, Opposite_Hai_34_list)
        Left_Hai_34_list = tileNumberTrans(game_info.N0, Left_Hai_34_list)
        self_score_34_List = decode_score_to_34(game_info.score[1])
        right_score_34_List = decode_score_to_34(game_info.score[2])
        Opposite_score_34_List = decode_score_to_34(game_info.score[3])
        Left_score_34_List = decode_score_to_34(game_info.score[0])
    elif player_pos == 2:
        self_Hai_34_List = tileNumberTrans(game_info.hai2, self_Hai_34_List)
        right_Hai_34_list = tileNumberTrans(game_info.N3, right_Hai_34_list)
        Opposite_Hai_34_list = tileNumberTrans(game_info.N0, Opposite_Hai_34_list)
        Left_Hai_34_list = tileNumberTrans(game_info.N1, Left_Hai_34_list)
        self_score_34_List = decode_score_to_34(game_info.score[2])
        right_score_34_List = decode_score_to_34(game_info.score[3])
        Opposite_score_34_List = decode_score_to_34(game_info.score[0])
        Left_score_34_List = decode_score_to_34(game_info.score[1])
    elif player_pos == 3:
        self_Hai_34_List = tileNumberTrans(game_info.hai3, self_Hai_34_List)
        right_Hai_34_list = tileNumberTrans(game_info.N0, right_Hai_34_list)
        Opposite_Hai_34_list = tileNumberTrans(game_info.N1, Opposite_Hai_34_list)
        Left_Hai_34_list = tileNumberTrans(game_info.N2, Left_Hai_34_list)
        self_score_34_List = decode_score_to_34(game_info.score[3])
        right_score_34_List = decode_score_to_34(game_info.score[0])
        Opposite_score_34_List = decode_score_to_34(game_info.score[1])
        Left_score_34_List = decode_score_to_34(game_info.score[2])
    
    self_Hai_136_List = decode_34_to_136(self_Hai_34_List)
    right_Hai_136_List = decode_34_to_136(right_Hai_34_list)
    Opposite_Hai_136_List = decode_34_to_136(Opposite_Hai_34_list)
    Left_Hai_136_list = decode_34_to_136(Left_Hai_34_list)
    Dora_Hai_136_list = decode_Dora_to_136(game_info.dora)
    Discard_Hai_136_list = decode_4playersDis_to_136(game_info)
    rounds_34_list = decode_rounds_to_34(game_info.rounds)
    
    game_info = cpoylist(game_info,0,self_Hai_136_List)
    game_info = cpoylist(game_info,136,right_Hai_136_List)
    game_info = cpoylist(game_info,272,Opposite_Hai_136_List)
    game_info = cpoylist(game_info,408,Left_Hai_136_list)
    game_info = cpoylist(game_info,544,Dora_Hai_136_list)
    game_info = cpoylist(game_info,680,Discard_Hai_136_list)
    game_info = cpoylist(game_info,816,self_score_34_List)
    game_info = cpoylist(game_info,850,right_score_34_List)
    game_info = cpoylist(game_info,884,Opposite_score_34_List)
    game_info = cpoylist(game_info,918,Left_score_34_List)
    game_info = cpoylist(game_info,952,rounds_34_list)

    game_info.chowFlag = 1
    return game_info

def record_chow_label(game_info: GameInfo, chow_Label:int):
    global chow_data_num
    if(game_info.chowFlag == 1):
        if(chow_Label == 0 and chow_data_num <= 0):
            return game_info
        elif(chow_Label == 0 and chow_data_num > 0):
            chow_data_num = chow_data_num - 1
        elif(chow_Label == 1):
            chow_data_num = chow_data_num + 1
        game_info.rowData[986] = chow_Label
        game_info.datalist.append(game_info.rowData)
        game_info.chowFlag = 0
    return game_info

def check_if_can_chow(Left_player_discard: int,hai:list):
    discard_34_type = int(Left_player_discard/4)
    if(discard_34_type >= 27):#超出萬餅條範圍
        return False
    elif((discard_34_type - 1) in hai and (discard_34_type - 2) in hai):#Left_player_discard最大的吃
        return True
    elif((discard_34_type - 1) in hai and (discard_34_type + 1) in hai):#Left_player_discard中間的吃
        return True
    elif((discard_34_type + 1) in hai and (discard_34_type + 2) in hai):#Left_player_discard最小的吃
        return True
    else:
        return False

def check_if_can_pung(player_discard: int,discard_player_pos: int,game_info: GameInfo):#如果沒人可以碰就回傳-1，如果有就回傳該玩家位置
    discard_34_type = int(player_discard/4)

    if(discard_player_pos != 0 and ((hai136_to_hai34(game_info.hai0).count(discard_34_type) - hai136_to_hai34(game_info.N0).count(discard_34_type)) >= 2)):#play0可以碰
        return 0
    elif(discard_player_pos != 1 and ((hai136_to_hai34(game_info.hai1).count(discard_34_type) - hai136_to_hai34(game_info.N1).count(discard_34_type)) >= 2)):#play1可以碰
        return 1
    elif(discard_player_pos != 2 and ((hai136_to_hai34(game_info.hai2).count(discard_34_type) - hai136_to_hai34(game_info.N2).count(discard_34_type)) >= 2)):#play2可以碰
        return 2
    elif(discard_player_pos != 3 and ((hai136_to_hai34(game_info.hai3).count(discard_34_type) - hai136_to_hai34(game_info.N3).count(discard_34_type)) >= 2)):#play3可以碰
        return 3
    else:
        return -1

def check_if_Discard_can_kong(player_discard: int,discard_player_pos: int,game_info: GameInfo):#如果沒人可以槓就回傳-1，如果有就回傳該玩家位置
    discard_34_type = int(player_discard/4)

    if(discard_player_pos != 0 and ((hai136_to_hai34(game_info.hai0).count(discard_34_type) - hai136_to_hai34(game_info.N0).count(discard_34_type)) >= 3)):#play0可以槓
        return 0
    elif(discard_player_pos != 1 and ((hai136_to_hai34(game_info.hai1).count(discard_34_type) - hai136_to_hai34(game_info.N1).count(discard_34_type)) >= 3)):#play1可以槓
        return 1
    elif(discard_player_pos != 2 and ((hai136_to_hai34(game_info.hai2).count(discard_34_type) - hai136_to_hai34(game_info.N2).count(discard_34_type)) >= 3)):#play2可以槓
        return 2
    elif(discard_player_pos != 3 and ((hai136_to_hai34(game_info.hai3).count(discard_34_type) - hai136_to_hai34(game_info.N3).count(discard_34_type)) >= 3)):#play3可以槓
        return 3
    else:
        return -1

def check_if_Draw_can_kong(player_draw: int,draw_player_pos: int,game_info: GameInfo):#如果沒人可以槓就回傳-1，如果有就回傳該玩家位置
    discard_34_type = int(player_draw/4)

    if(draw_player_pos != 0 and ((hai136_to_hai34(game_info.hai0).count(discard_34_type) - hai136_to_hai34(game_info.N0).count(discard_34_type)) >= 3)):#play0可以槓
        return 0
    elif(draw_player_pos != 1 and ((hai136_to_hai34(game_info.hai1).count(discard_34_type) - hai136_to_hai34(game_info.N1).count(discard_34_type)) >= 3)):#play1可以槓
        return 1
    elif(draw_player_pos != 2 and ((hai136_to_hai34(game_info.hai2).count(discard_34_type) - hai136_to_hai34(game_info.N2).count(discard_34_type)) >= 3)):#play2可以槓
        return 2
    elif(draw_player_pos != 3 and ((hai136_to_hai34(game_info.hai3).count(discard_34_type) - hai136_to_hai34(game_info.N3).count(discard_34_type)) >= 3)):#play3可以槓
        return 3
    else:
        return -1

def hai136_to_hai34(hai136: list):
    hai34 = []

    for hai in hai136:
        hai34.append(int(hai/4))
    return hai34

def record_pung_info(discard:int, player_pos: int,game_info: GameInfo):#根據check_if_can_pung位置處理，如果是-1就不處理
    if player_pos == -1:
        return game_info
    game_info.rowData = [0]*features_nums
    self_Hai_34_List = [0] * 34
    right_Hai_34_list = [0] * 34
    Opposite_Hai_34_list = [0] * 34
    Left_Hai_34_list = [0] * 34

    if player_pos == 0:
        self_Hai_34_List = tileNumberTrans(game_info.hai0, self_Hai_34_List)
        right_Hai_34_list = tileNumberTrans(game_info.N1, right_Hai_34_list)
        Opposite_Hai_34_list = tileNumberTrans(game_info.N2, Opposite_Hai_34_list)
        Left_Hai_34_list = tileNumberTrans(game_info.N3, Left_Hai_34_list)
        self_score_34_List = decode_score_to_34(game_info.score[0])
        right_score_34_List = decode_score_to_34(game_info.score[1])
        Opposite_score_34_List = decode_score_to_34(game_info.score[2])
        Left_score_34_List = decode_score_to_34(game_info.score[3])
    elif player_pos == 1:
        self_Hai_34_List = tileNumberTrans(game_info.hai1, self_Hai_34_List)
        right_Hai_34_list = tileNumberTrans(game_info.N2, right_Hai_34_list)
        Opposite_Hai_34_list = tileNumberTrans(game_info.N3, Opposite_Hai_34_list)
        Left_Hai_34_list = tileNumberTrans(game_info.N0, Left_Hai_34_list)
        self_score_34_List = decode_score_to_34(game_info.score[1])
        right_score_34_List = decode_score_to_34(game_info.score[2])
        Opposite_score_34_List = decode_score_to_34(game_info.score[3])
        Left_score_34_List = decode_score_to_34(game_info.score[0])
    elif player_pos == 2:
        self_Hai_34_List = tileNumberTrans(game_info.hai2, self_Hai_34_List)
        right_Hai_34_list = tileNumberTrans(game_info.N3, right_Hai_34_list)
        Opposite_Hai_34_list = tileNumberTrans(game_info.N0, Opposite_Hai_34_list)
        Left_Hai_34_list = tileNumberTrans(game_info.N1, Left_Hai_34_list)
        self_score_34_List = decode_score_to_34(game_info.score[2])
        right_score_34_List = decode_score_to_34(game_info.score[3])
        Opposite_score_34_List = decode_score_to_34(game_info.score[0])
        Left_score_34_List = decode_score_to_34(game_info.score[1])
    elif player_pos == 3:
        self_Hai_34_List = tileNumberTrans(game_info.hai3, self_Hai_34_List)
        right_Hai_34_list = tileNumberTrans(game_info.N0, right_Hai_34_list)
        Opposite_Hai_34_list = tileNumberTrans(game_info.N1, Opposite_Hai_34_list)
        Left_Hai_34_list = tileNumberTrans(game_info.N2, Left_Hai_34_list)
        self_score_34_List = decode_score_to_34(game_info.score[3])
        right_score_34_List = decode_score_to_34(game_info.score[0])
        Opposite_score_34_List = decode_score_to_34(game_info.score[1])
        Left_score_34_List = decode_score_to_34(game_info.score[2])
    
    self_Hai_136_List = decode_34_to_136(self_Hai_34_List)
    right_Hai_136_List = decode_34_to_136(right_Hai_34_list)
    Opposite_Hai_136_List = decode_34_to_136(Opposite_Hai_34_list)
    Left_Hai_136_list = decode_34_to_136(Left_Hai_34_list)
    Dora_Hai_136_list = decode_Dora_to_136(game_info.dora)
    Discard_Hai_136_list = decode_4playersDis_to_136(game_info)
    rounds_34_list = decode_rounds_to_34(game_info.rounds)
    
    game_info = cpoylist(game_info,0,self_Hai_136_List)
    game_info = cpoylist(game_info,136,right_Hai_136_List)
    game_info = cpoylist(game_info,272,Opposite_Hai_136_List)
    game_info = cpoylist(game_info,408,Left_Hai_136_list)
    game_info = cpoylist(game_info,544,Dora_Hai_136_list)
    game_info = cpoylist(game_info,680,Discard_Hai_136_list)
    game_info = cpoylist(game_info,816,self_score_34_List)
    game_info = cpoylist(game_info,850,right_score_34_List)
    game_info = cpoylist(game_info,884,Opposite_score_34_List)
    game_info = cpoylist(game_info,918,Left_score_34_List)
    game_info = cpoylist(game_info,952,rounds_34_list)
    game_info.pungFlag = 1
    return game_info

def record_pung_label(game_info: GameInfo, pung_Label:int):
    global pung_data_num
    if(game_info.pungFlag == 1):
        if(pung_Label == 0 and pung_data_num <= 0):
            return game_info
        elif(pung_Label == 0 and pung_data_num > 0):
            pung_data_num = pung_data_num - 1
        elif(pung_Label == 1):
            pung_data_num = pung_data_num + 1
        game_info.rowData[986] = pung_Label
        game_info.datalist.append(game_info.rowData)
        game_info.pungFlag = 0
    return game_info

def record_kong_info(discard:int, player_pos: int,game_info: GameInfo):#根據check_if_can_kong位置處理，如果是-1就不處理
    if player_pos == -1:
        return game_info
    game_info.rowData = [0]*features_nums
    self_Hai_34_List = [0] * 34
    right_Hai_34_list = [0] * 34
    Opposite_Hai_34_list = [0] * 34
    Left_Hai_34_list = [0] * 34

    if player_pos == 0:
        self_Hai_34_List = tileNumberTrans(game_info.hai0, self_Hai_34_List)
        right_Hai_34_list = tileNumberTrans(game_info.N1, right_Hai_34_list)
        Opposite_Hai_34_list = tileNumberTrans(game_info.N2, Opposite_Hai_34_list)
        Left_Hai_34_list = tileNumberTrans(game_info.N3, Left_Hai_34_list)
        self_score_34_List = decode_score_to_34(game_info.score[0])
        right_score_34_List = decode_score_to_34(game_info.score[1])
        Opposite_score_34_List = decode_score_to_34(game_info.score[2])
        Left_score_34_List = decode_score_to_34(game_info.score[3])
    elif player_pos == 1:
        self_Hai_34_List = tileNumberTrans(game_info.hai1, self_Hai_34_List)
        right_Hai_34_list = tileNumberTrans(game_info.N2, right_Hai_34_list)
        Opposite_Hai_34_list = tileNumberTrans(game_info.N3, Opposite_Hai_34_list)
        Left_Hai_34_list = tileNumberTrans(game_info.N0, Left_Hai_34_list)
        self_score_34_List = decode_score_to_34(game_info.score[1])
        right_score_34_List = decode_score_to_34(game_info.score[2])
        Opposite_score_34_List = decode_score_to_34(game_info.score[3])
        Left_score_34_List = decode_score_to_34(game_info.score[0])
    elif player_pos == 2:
        self_Hai_34_List = tileNumberTrans(game_info.hai2, self_Hai_34_List)
        right_Hai_34_list = tileNumberTrans(game_info.N3, right_Hai_34_list)
        Opposite_Hai_34_list = tileNumberTrans(game_info.N0, Opposite_Hai_34_list)
        Left_Hai_34_list = tileNumberTrans(game_info.N1, Left_Hai_34_list)
        self_score_34_List = decode_score_to_34(game_info.score[2])
        right_score_34_List = decode_score_to_34(game_info.score[3])
        Opposite_score_34_List = decode_score_to_34(game_info.score[0])
        Left_score_34_List = decode_score_to_34(game_info.score[1])
    elif player_pos == 3:
        self_Hai_34_List = tileNumberTrans(game_info.hai3, self_Hai_34_List)
        right_Hai_34_list = tileNumberTrans(game_info.N0, right_Hai_34_list)
        Opposite_Hai_34_list = tileNumberTrans(game_info.N1, Opposite_Hai_34_list)
        Left_Hai_34_list = tileNumberTrans(game_info.N2, Left_Hai_34_list)
        self_score_34_List = decode_score_to_34(game_info.score[3])
        right_score_34_List = decode_score_to_34(game_info.score[0])
        Opposite_score_34_List = decode_score_to_34(game_info.score[1])
        Left_score_34_List = decode_score_to_34(game_info.score[2])
    
    self_Hai_136_List = decode_34_to_136(self_Hai_34_List)
    right_Hai_136_List = decode_34_to_136(right_Hai_34_list)
    Opposite_Hai_136_List = decode_34_to_136(Opposite_Hai_34_list)
    Left_Hai_136_list = decode_34_to_136(Left_Hai_34_list)
    Dora_Hai_136_list = decode_Dora_to_136(game_info.dora)
    Discard_Hai_136_list = decode_4playersDis_to_136(game_info)
    rounds_34_list = decode_rounds_to_34(game_info.rounds)
    
    game_info = cpoylist(game_info,0,self_Hai_136_List)
    game_info = cpoylist(game_info,136,right_Hai_136_List)
    game_info = cpoylist(game_info,272,Opposite_Hai_136_List)
    game_info = cpoylist(game_info,408,Left_Hai_136_list)
    game_info = cpoylist(game_info,544,Dora_Hai_136_list)
    game_info = cpoylist(game_info,680,Discard_Hai_136_list)
    game_info = cpoylist(game_info,816,self_score_34_List)
    game_info = cpoylist(game_info,850,right_score_34_List)
    game_info = cpoylist(game_info,884,Opposite_score_34_List)
    game_info = cpoylist(game_info,918,Left_score_34_List)
    game_info = cpoylist(game_info,952,rounds_34_list)
    game_info.kongFlag = 1
    return game_info

def record_kong_label(game_info: GameInfo, kong_Label:int):
    global kong_data_num
    if(game_info.kongFlag == 1):
        if(kong_Label == 0 and kong_data_num <= 0):
            return game_info
        elif(kong_Label == 0 and kong_data_num > 0):
            kong_data_num = kong_data_num - 1
        elif(kong_Label == 1):
            kong_data_num = kong_data_num + 1
        game_info.rowData[986] = kong_Label
        game_info.datalist.append(game_info.rowData)
        game_info.kongFlag = 0
    return game_info

def decode_34_to_136(Hai_34_List: list):
    Hai_136_List = [0]*136

    for i in range(4):
        for j in range(34):
            if(Hai_34_List[j] > 0):
                Hai_136_List[i*34+j] = 1
                Hai_34_List[j] = Hai_34_List[j] - 1
    return Hai_136_List

def decode_Dora_to_136(Dora_Hai_5_list: list):
    Dora_Hai_136_list = [0]*136
    for i in range(5):
        if Dora_Hai_5_list[i] == -1:
            continue
        else:
            doraHai = Dora_Hai_5_list[i]
            while(Dora_Hai_136_list[doraHai] == 1):
                doraHai = doraHai + 34
            Dora_Hai_136_list[doraHai] = 1
    return Dora_Hai_136_list

def decode_Dora_to_34(Dora_Hai_5_list: list):
    Dora_Hai_34_list = [0]*34
    for i in range(5):
        if Dora_Hai_5_list[i] == -1:
            continue
        else:
            Dora_Hai_34_list[Dora_Hai_5_list[i]] = 1
    return Dora_Hai_34_list

def decode_4playersDis_to_136(game_info: GameInfo):
    Discard_Hai_136_list = [0]*136
    for dis in game_info.play0Dis:
        dis34 = int(dis/4)
        while(Discard_Hai_136_list[dis34] == 1):
            dis34 = dis34 + 34
        Discard_Hai_136_list[dis34] = 1
    for dis in game_info.play1Dis:
        dis34 = int(dis/4)
        while(Discard_Hai_136_list[dis34] == 1):
            dis34 = dis34 + 34
        Discard_Hai_136_list[dis34] = 1
    for dis in game_info.play2Dis:
        dis34 = int(dis/4)
        while(Discard_Hai_136_list[dis34] == 1):
            dis34 = dis34 + 34
        Discard_Hai_136_list[dis34] = 1
    for dis in game_info.play3Dis:
        dis34 = int(dis/4)
        while(Discard_Hai_136_list[dis34] == 1):
            dis34 = dis34 + 34
        Discard_Hai_136_list[dis34] = 1
    return Discard_Hai_136_list

def decode_3playersDis_to_102(game_info: GameInfo, random_player: int):
    Discard_Hai_102_list = [0]*102

    all_discards = [game_info.play0Dis, game_info.play1Dis, game_info.play2Dis, game_info.play3Dis]
    ordered_players = [(random_player + 1) % 4,(random_player + 2) % 4,(random_player + 3) % 4]

    for seg_idx, player in enumerate(ordered_players):
        base = seg_idx * 34  # 第幾段 (0, 34, 68)
        recent_discards = all_discards[player][-6:]  # 該玩家最後 6 張棄牌

        for dis in recent_discards:
            dis34 = int(dis / 4)
            target_idx = base + dis34  # 將該牌標記到該玩家的區段中

            if target_idx < len(Discard_Hai_102_list):
                Discard_Hai_102_list[target_idx] = 1

    return Discard_Hai_102_list

def calc_shanten_change(hand_tile:list, discard:int):
    shanten_calculator = Shanten()
    origin_shanten = shanten_calculator.calculate_shanten(decode_to_tile34(hand_tile))

    hand_tile_copy = hand_tile[:]
    hand_tile_copy.append(int(discard/4))

    shanten_change_34 = [0]*34

    for i in range(34):
        if i in hand_tile_copy:
            hand_tile_copy.remove(i)
            new_shanten = shanten_calculator.calculate_shanten(decode_to_tile34(hand_tile_copy))
            shanten_change_34[i] = new_shanten - origin_shanten
            hand_tile_copy.append(i)
        else:
            shanten_change_34[i] = 0

    return shanten_change_34

def calc_pon_potential(hand_tile:list,remain_count:list):
    pon_potential = [0]*34

    hand_tile_count = decode_to_tile34(hand_tile)

    for i in range(34):
        if hand_tile_count[i]==3:#已有pon
            pon_potential[i] = 1
        elif hand_tile_count[i] == 2 and remain_count[i] >= 1:#已有pair，且還有剩
            pon_potential[i] = 0.7
        elif hand_tile_count[i] == 1 and remain_count[i] >= 2:#只有單張，且還有剩
            pon_potential[i] = 0.4
        elif hand_tile_count[i] == 0 and remain_count[i] >= 3:#只有單張，且還有剩
            pon_potential[i] = 0.1
        else:
            pon_potential[i] = 0
    return pon_potential
        
def calc_chi_potential(hand_tile:list,remain_count:list):
    chi_potential_34 = [0]*34

    for i in range(27):
        point = 0.0
        if i % 9 >=2:#(i-2.i-1.i)組合
            if i-1 in hand_tile:
                point += 1
            else:
                point += remain_count[i-1] * 0.1

            if i-2 in hand_tile:
                point += 0.7
            else:
                point += remain_count[i-2] * 0.05

        if i % 9 >=1 and i % 9 <= 7:#(i-1.i.i+1)組合
            if i-1 in hand_tile:
                point += 1
            else:
                point += remain_count[i-1] * 0.1

            if i+1 in hand_tile:
                point += 1
            else:
                point += remain_count[i+1] * 0.1

        if i % 9 <=6:#(i.i+1.i+2)組合
            if i+1 in hand_tile:
                point += 1
            else:
                point += remain_count[i+1] * 0.1

            if i+2 in hand_tile:
                point += 0.7
            else:
                point += remain_count[i+2] * 0.05
    
        chi_potential_34[i] = point

    return chi_potential_34

def decode_to_tile34(hand_tiles34):
        tile34 = [0]*34
        for tile in hand_tiles34:
            tile34[tile] += 1
        return tile34

def decode_score_to_34(score: int):
    score_34 = [0]*34
    index = int(score/20)
    if index < 0:
        index = 0
    elif index >=34:
        index = 33
    score_34[index] = 1
    return score_34

def decode_rounds_to_34(round:int):
    rounds_34 = [0]*34
    index = int((69 - round)/2)
    if index > 33:
        index = 33
    rounds_34[index] = 1
    return rounds_34

def check_if_can_Riichi(player_hai: list, player_Open_Meld: list):#傳入手牌及副露，回傳是否可以立直
    if player_Open_Meld != []:
        return False

    unique_tiles = set(player_hai)
    for tile in unique_tiles:
        new_hai = player_hai.copy()
        new_hai.remove(tile)

        if is_tenpai(new_hai):
            return True

def is_tenpai(hai: list):#傳入手牌，回傳是否可以聽牌

    #依序加入可能的摸牌
    for i in range(34):
        if hai.count(i) < 4:#確保摸進來不超過4
            new_hai = hai.copy()
            new_hai.append(i)

            hai_34count = return_hai34_count(new_hai)

            for tile_num_index in range(len(hai_34count)):#依序取眼
                if tile_num_index >= 2:
                    take_eye_hai_34count = hai_34count.copy()
                    take_eye_hai_34count[tile_num_index] -= 2

                    if can_four(take_eye_hai_34count):
                        return True
    return False


def return_hai34_count(hai: list):#回傳每種牌的數量
    hai34 = [0]*34
    for tile in hai:
        hai34[tile] = hai34[tile] + 1
    return hai34

def can_four(take_eye_hai_34count: list):#傳入已取眼的牌數量，回傳是否可以湊四搭
    if take_eye_hai_34count == []:
        return True
    elif take_eye_hai_34count[0] == 0:
        return can_four(take_eye_hai_34count[1:])
    elif take_eye_hai_34count[0] >= 3:#取賴
        take_eye_hai_34count[0] -= 3
        return can_four(take_eye_hai_34count)
    elif len(take_eye_hai_34count) >= 3:#確認是否可以取順
        if take_eye_hai_34count[0] >= 1 and take_eye_hai_34count[1] >= 1 and take_eye_hai_34count[2] >= 1:
            take_eye_hai_34count[0] -= 1
            take_eye_hai_34count[1] -= 1
            take_eye_hai_34count[2] -= 1
            return can_four(take_eye_hai_34count)
        else:
            return False
    else:
        return False
        
def record_riichi_info(game_info: GameInfo,player_pos: int):
    game_info.rowData = [0]*features_nums
    self_Hai_34_List = [0] * 34
    right_Hai_34_list = [0] * 34
    Opposite_Hai_34_list = [0] * 34
    Left_Hai_34_list = [0] * 34

    if player_pos == 0:
        self_Hai_34_List = tileNumberTrans(game_info.hai0, self_Hai_34_List)
        right_Hai_34_list = tileNumberTrans(game_info.N1, right_Hai_34_list)
        Opposite_Hai_34_list = tileNumberTrans(game_info.N2, Opposite_Hai_34_list)
        Left_Hai_34_list = tileNumberTrans(game_info.N3, Left_Hai_34_list)
        self_score_34_List = decode_score_to_34(game_info.score[0])
        right_score_34_List = decode_score_to_34(game_info.score[1])
        Opposite_score_34_List = decode_score_to_34(game_info.score[2])
        Left_score_34_List = decode_score_to_34(game_info.score[3])
    elif player_pos == 1:
        self_Hai_34_List = tileNumberTrans(game_info.hai1, self_Hai_34_List)
        right_Hai_34_list = tileNumberTrans(game_info.N2, right_Hai_34_list)
        Opposite_Hai_34_list = tileNumberTrans(game_info.N3, Opposite_Hai_34_list)
        Left_Hai_34_list = tileNumberTrans(game_info.N0, Left_Hai_34_list)
        self_score_34_List = decode_score_to_34(game_info.score[1])
        right_score_34_List = decode_score_to_34(game_info.score[2])
        Opposite_score_34_List = decode_score_to_34(game_info.score[3])
        Left_score_34_List = decode_score_to_34(game_info.score[0])
    elif player_pos == 2:
        self_Hai_34_List = tileNumberTrans(game_info.hai2, self_Hai_34_List)
        right_Hai_34_list = tileNumberTrans(game_info.N3, right_Hai_34_list)
        Opposite_Hai_34_list = tileNumberTrans(game_info.N0, Opposite_Hai_34_list)
        Left_Hai_34_list = tileNumberTrans(game_info.N1, Left_Hai_34_list)
        self_score_34_List = decode_score_to_34(game_info.score[2])
        right_score_34_List = decode_score_to_34(game_info.score[3])
        Opposite_score_34_List = decode_score_to_34(game_info.score[0])
        Left_score_34_List = decode_score_to_34(game_info.score[1])
    elif player_pos == 3:
        self_Hai_34_List = tileNumberTrans(game_info.hai3, self_Hai_34_List)
        right_Hai_34_list = tileNumberTrans(game_info.N0, right_Hai_34_list)
        Opposite_Hai_34_list = tileNumberTrans(game_info.N1, Opposite_Hai_34_list)
        Left_Hai_34_list = tileNumberTrans(game_info.N2, Left_Hai_34_list)
        self_score_34_List = decode_score_to_34(game_info.score[3])
        right_score_34_List = decode_score_to_34(game_info.score[0])
        Opposite_score_34_List = decode_score_to_34(game_info.score[1])
        Left_score_34_List = decode_score_to_34(game_info.score[2])
    
    self_Hai_136_List = decode_34_to_136(self_Hai_34_List)
    right_Hai_136_List = decode_34_to_136(right_Hai_34_list)
    Opposite_Hai_136_List = decode_34_to_136(Opposite_Hai_34_list)
    Left_Hai_136_list = decode_34_to_136(Left_Hai_34_list)
    Dora_Hai_136_list = decode_Dora_to_136(game_info.dora)
    Discard_Hai_136_list = decode_4playersDis_to_136(game_info)
    rounds_34_list = decode_rounds_to_34(game_info.rounds)
    
    game_info = cpoylist(game_info,0,self_Hai_136_List)
    game_info = cpoylist(game_info,136,right_Hai_136_List)
    game_info = cpoylist(game_info,272,Opposite_Hai_136_List)
    game_info = cpoylist(game_info,408,Left_Hai_136_list)
    game_info = cpoylist(game_info,544,Dora_Hai_136_list)
    game_info = cpoylist(game_info,680,Discard_Hai_136_list)
    game_info = cpoylist(game_info,816,self_score_34_List)
    game_info = cpoylist(game_info,850,right_score_34_List)
    game_info = cpoylist(game_info,884,Opposite_score_34_List)
    game_info = cpoylist(game_info,918,Left_score_34_List)
    game_info = cpoylist(game_info,952,rounds_34_list)

    game_info.riichiFlag = 1
    return game_info

def record_riichi_label(game_info: GameInfo, riichi_Label:int):
    global riichi_data_num
    if(game_info.riichiFlag == 1):
        if(riichi_Label == 0 and riichi_data_num <= 0):
            return game_info
        elif(riichi_Label == 0 and riichi_data_num > 0):
            riichi_data_num = riichi_data_num - 1
        elif(riichi_Label == 1):
            riichi_data_num = riichi_data_num + 1
        game_info.rowData[986] = riichi_Label
        game_info.datalist.append(game_info.rowData)
        game_info.riichiFlag = 0
    return game_info

In [160]:
"""
將牌局資料轉成csv形式
"""
for year in range(2021,2023):
    cur_game = 0
    input_folder = f'E:/專題/data/{year}/scc game'
    with open(f'E:/專題/data/{year}/{writed_file}', mode='w', newline='', encoding='utf-8') as file:
        #處理每個features的名稱
        writer = csv.writer(file)
        data_frame = pd.read_excel(f'E:/專題/model_feature/{feature_Name}', header=None)
        names = data_frame.iloc[0, :features_nums].values #提取的標籤數量
        pd.DataFrame([names]).to_csv(file, header=False, index=False)
        #遍歷scc game裡的txt
        for filename in os.listdir(input_folder):
            cur_game = cur_game + 1
            if cur_game >= record_games:
                break
            txt_path = os.path.join(input_folder, filename)
            random_Player = 0 #random.randint(0,3)#隨機取一位玩家當作主視角
            #開始處理scc_game.txt
            with open(txt_path, 'r', encoding='utf-8') as file:
                #依<>切分txt並儲存成list
                content = file.read()
                conlist = re.split(r'[<>]', content)
                conlist = [s for s in conlist if s]
                del conlist[1]
                #再將/刪除
                infolist = []
                for item in conlist:
                    sub = re.split(r'[ /]', item)
                    for ele in sub:
                        infolist.append(ele)
                infolist = [s for s in infolist if s]

                game_info = GameInfo() #初始化牌局資訊
                #開始遍歷牌局訊息並處理
                for element in infolist:
                    if(element in jumpWords):#不明原因在打牌階段出現的詞: 
                        continue
                    elif(element == 'INIT'):
                        game_info.reset()
                        game_info.eventState = EventState.INIT_DORA
                    elif(game_info.eventState == EventState.INIT_DORA):
                        game_info = INIT_Dora(game_info,element)
                    elif(game_info.playState == PlayState.INIT and 'oya' in element):
                        game_info = Switch_StartPlayer_And_Selfwind(game_info,element)
                    elif(game_info.playState == PlayState.INIT and 'ten' in element):
                        game_info = Process_Score(game_info,element)
                    elif('hai0' in element or 'hai1' in element or 'hai2' in element or 'hai3' in element):
                        game_info = INIT_Hai(game_info,element,random_Player)#處理完第四家手牌順便設置playState = PLAYING
                    elif(game_info.playState == PlayState.PLAYING):
                        game_info = Process_Playing(game_info, element, random_Player)
                    elif(game_info.playState == PlayState.WIN):
                        game_info = Process_Win(game_info, element, random_Player, writer)
                    elif(game_info.playState == PlayState.DRAW):
                        game_info = Process_Draw(game_info, element, random_Player, writer)

1011101001101010 0
1100010001001001 0
1100001001001011 0
1100011001101011 2
1011110000001011 0
0101001001101011 1
1011101000001001 3
0011110000001001 1
1001001001001001 1
1010011000101001 0
0000110000001001 0
1010011001101001 0


AssertionError: Too many tiles = 15